# Reto 1

Considere el conjunto de datos FraudeCanastas.csv disponible en el archivo correspondiente.

# Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# For logistic regression
from tensorflow.keras.layers import Dense
# For autoencoders and deep neural networks
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from keras.initializers import VarianceScaling
from keras.optimizers import SGD
# Training data
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score

ModuleNotFoundError: No module named 'pandas'

# Lectura y carga de datos

Primero vamos a descomprimir los archivos con los que vamos a trabajar, posteriormente vamos a leer los datos y cargarlos en un dataframe

In [3]:
# !unzip /content/drive/MyDrive/Bourbaki/FraudeCanastas.zip -d /content/drive/MyDrive/Bourbaki/

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Bourbaki/FraudeCanastas.csv')

In [5]:
df.head(5)

,ID,APPLE PRODUCTDESCRIPTION | SAMSUNG | MODEL90,AUDIO ACCESSORIES | AB AUDIO | AB AUDIO GO AIR TRUE WIRELESS BLUETOOTH IN-EAR H,AUDIO ACCESSORIES | APPLE | 2019 APPLE AIRPODS WITH CHARGING CASE,AUDIO ACCESSORIES | APPLE | 2019 APPLE AIRPODS WITH CHARGING CASE 2ND GENERATI,AUDIO ACCESSORIES | APPLE | 2019 APPLE AIRPODS WITH WIRELESS CHARGING CASE,AUDIO ACCESSORIES | APPLE | 2019 APPLE AIRPODS WITH WIRELESS CHARGING CASE 2ND,AUDIO ACCESSORIES | APPLE | 2021 APPLE AIRPODS WITH MAGSAFE CHARGING CASE 3RD,AUDIO ACCESSORIES | APPLE | AIRPODS PRO,AUDIO ACCESSORIES | APPLE | APPLE AIRPODS MAX,...,WOMEN S NIGHTWEAR | ANYDAY RETAILER | ANYDAY RETAILER LEOPARD PRINT JERSEY PY,WOMEN S NIGHTWEAR | RETAILER | RETAILER CLEO VELOUR JOGGER LOUNGE PANT,WOMEN S NIGHTWEAR | SOSANDAR | SOSANDAR ZEBRA PRINT PYJAMA BOTTOMS BLACK 10,Nb_of_items,total_of_items,costo_total,costo_medio_item,costo_item_max,costo_item_min,fraud_flag
0,130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,2,1299,649.500000,1299,0.0,1.0
1,195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3,3,4119,1373.000000,2470,0.0,1.0
2,217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,2,2806,1403.000000,2799,7.0,1.0
3,552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,2,1206,603.000000,1199,7.0,1.0
4,854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,19,27,1807,66.925926,195,4.0,1.0


Ahora voy a graficar los histogramas de las fraud flags de el dataframe df

Matriz de correlación

# Preprocesamiento

Primero separamos las características (X) y la variable objetivo (y)

In [6]:
X = df.drop(['fraud_flag', 'ID'], axis=1)
y = df['fraud_flag']

Luego escalamos las características usando MinMaxScaler, esto porque las escalas de las características varían mucho entre sí

In [7]:
scaler = MinMaxScaler()
X_std = scaler.fit_transform(X)
X_std = pd.DataFrame(X_std, columns=X.columns)
X_std.head(5)

,APPLE PRODUCTDESCRIPTION | SAMSUNG | MODEL90,AUDIO ACCESSORIES | AB AUDIO | AB AUDIO GO AIR TRUE WIRELESS BLUETOOTH IN-EAR H,AUDIO ACCESSORIES | APPLE | 2019 APPLE AIRPODS WITH CHARGING CASE,AUDIO ACCESSORIES | APPLE | 2019 APPLE AIRPODS WITH CHARGING CASE 2ND GENERATI,AUDIO ACCESSORIES | APPLE | 2019 APPLE AIRPODS WITH WIRELESS CHARGING CASE,AUDIO ACCESSORIES | APPLE | 2019 APPLE AIRPODS WITH WIRELESS CHARGING CASE 2ND,AUDIO ACCESSORIES | APPLE | 2021 APPLE AIRPODS WITH MAGSAFE CHARGING CASE 3RD,AUDIO ACCESSORIES | APPLE | AIRPODS PRO,AUDIO ACCESSORIES | APPLE | APPLE AIRPODS MAX,AUDIO ACCESSORIES | APPLE | APPLE AIRPODS MAX NOISE CANCELLING WIRELESS BLUETO,...,WOMEN S FOOTWEAR | UGG | UGG CLASSIC MINI II SHORT SHEEPSKIN BOOTS CHESTNUT,WOMEN S NIGHTWEAR | ANYDAY RETAILER | ANYDAY RETAILER LEOPARD PRINT JERSEY PY,WOMEN S NIGHTWEAR | RETAILER | RETAILER CLEO VELOUR JOGGER LOUNGE PANT,WOMEN S NIGHTWEAR | SOSANDAR | SOSANDAR ZEBRA PRINT PYJAMA BOTTOMS BLACK 10,Nb_of_items,total_of_items,costo_total,costo_medio_item,costo_item_max,costo_item_min
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.027027,0.023256,0.044386,0.030459,0.058835,0.00000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.054054,0.046512,0.160283,0.066711,0.117771,0.00000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.027027,0.023256,0.106321,0.068215,0.134330,0.00035
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.027027,0.023256,0.040564,0.028129,0.053802,0.00035
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.486486,0.604651,0.065264,0.001268,0.003271,0.00020


Después dividimos los datos en conjuntos de entrenamiento y prueba

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# Autoencoder

Primero definimos la arquitectura del modelo

In [31]:
def autoencoder(dims,act='relu', init='glorot_uniform'):
    """
    Modelo de autoencoder simétrico completamente conectado.

    dims: lista de los tamaños de las capas del encoder como [500, 500, 2000, 10].
dims[0] es la dimensión de entrada, dims[-1] es el tamaño de la capa oculta latente.

    act: función de activación

    return:
(autoencoder_model, encoder_model): modelo del autoencoder y modelo del encoder
    """

    n_stacks = len(dims) - 1

    input_data = Input(shape=(dims[0],), name='input')
    x = input_data

    # internal layers of encoder
    for i in range(n_stacks-1):
        x = Dense(dims[i + 1], activation=act,  kernel_initializer=init, name='encoder_%d' % i)(x)
    # latent hidden layer
    encoded = Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(x)
    x = encoded
    # internal layers of decoder
    for i in range(n_stacks-1, 0, -1):
        x = Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(x)
    # decoder output
    x = Dense(dims[0], kernel_initializer=init, name='decoder_0')(x)

    decoded = x
    autoencoder_model = Model(inputs=input_data, outputs=decoded, name='autoencoder')
    encoder_model     = Model(inputs=input_data, outputs=encoded, name='encoder')

    return autoencoder_model, encoder_model

In [29]:
n_epochs   = 100
batch_size = 128
dims = [X.shape[-1], 500, 500, 2000, 10]
## VarianceScaling
### Es una estrategia de inicialización de peso que se utiliza para ayudar con el entrenamiento de redes neuronales.
### Escala los pesos iniciales de las capas en función de la varianza de las entradas, lo que
### puede ayudar a mantener un flujo de gradiente saludable a través de la red y potencialmente mejorar el rendimiento del entrenamiento.
init_ = VarianceScaling(scale=1. / 3., mode='fan_in',
                           distribution='uniform')
pretrain_optimizer = SGD(learning_rate=0.1, momentum=0.9)

In [32]:
pretrain_epochs = n_epochs
batch_size = batch_size
autoencoder, encoder = autoencoder(dims, init=init_)

In [33]:
autoencoder.summary()

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)                   │ (None, 2455)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_0 (Dense)                    │ (None, 500)                 │       1,228,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_1 (Dense)                    │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_2 (Dense)                    │ (None, 2000)                │       1,002,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder_3 (Dense)                    │ (None, 10)                  │          20,010 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder_3 (Dense)                    │ (None, 2000)                │          22,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder_2 (Dense)                    │ (None, 500)                 │       1,000,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder_1 (Dense)                    │ (None, 500)                 │         250,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder_0 (Dense)                    │ (None, 2455)                │       1,229,955 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,003,465 (19.09 MB)

 Trainable params: 5,003,465 (19.09 MB)

 Non-trainable params: 0 (0.00 B)

Ahora vamos a extraer las características del encoder

In [ ]:
autoencoder.compile(optimizer=pretrain_optimizer, loss='mse')
autoencoder.fit(X,X, batch_size=batch_size, epochs=pretrain_epochs)

Epoch 1/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 12s 147ms/step - loss: nan
Epoch 2/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 9s 122ms/step - loss: nan
Epoch 3/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 12s 142ms/step - loss: nan
Epoch 4/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 20s 136ms/step - loss: nan
Epoch 5/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 10s 126ms/step - loss: nan
Epoch 6/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 10s 143ms/step - loss: nan
Epoch 7/100
 5/73 ━━━━━━━━━━━━━━━━━━━━ 8s 123ms/step - loss: nan

Transformamos los datos de entrenamiento y prueba

In [66]:
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)

233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


Entrenamos la regresión logística

In [67]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train_encoded, y_train)

LogisticRegression()

## Evaluación del modelo

In [68]:
y_pred = logistic_model.predict(X_test_encoded)

Finalmente vamos a calcular las métricas de evaluación

In [16]:
# prompt: Calcula algunas métricas de evaluación útiles para mi regresión logística



# ... (Your existing code) ...

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{conf_matrix}")

# Calcular el puntaje F1
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1}")

# Calcular el AUC-ROC (Area Under the ROC Curve)
roc_auc = roc_auc_score(y_test, logistic_model.predict_proba(X_test_encoded)[:, 1])
print(f"ROC AUC Score: {roc_auc}")



NameError: name 'y_pred' is not defined

# Red neuronal profunda

Ahora voy a entrenar una red neuronal
profunda para clasificar fraude. Empezaremos definiendo la arquitectura del modelo

In [15]:
# prompt: Define la arquitectura de una red neuronal profunda para clasificar fraude en mi dataframe

# Define the deep neural network architecture
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),  # Input layer with the encoded features
    layers.Dense(64, activation='relu'),  # Hidden layer with 64 neurons and ReLU activation
    layers.Dropout(0.2),  # Dropout layer for regularization
    layers.Dense(32, activation='relu'),  # Another hidden layer with 32 neurons and ReLU activation
    layers.Dropout(0.2),  # Dropout layer for regularization
    layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)



Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.7891 - loss: 0.6082 - val_accuracy: 0.8558 - val_loss: 0.3890
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8626 - loss: 0.3623 - val_accuracy: 0.8605 - val_loss: 0.3553
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8795 - loss: 0.2934 - val_accuracy: 0.8625 - val_loss: 0.3925
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8843 - loss: 0.2877 - val_accuracy: 0.8585 - val_loss: 0.3903
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8882 - loss: 0.2665 - val_accuracy: 0.8605 - val_loss: 0.4024
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8935 - loss: 0.2531 - val_accuracy: 0.8605 - val_loss: 0.4061
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8940 - loss: 0.2544 - val_accuracy: 0.8618 - val_loss: 0.4060
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8948 - loss: 0.2454 - val_accuracy: 0

NameError: name 'X_test_encoded' is not defined

Ahora evaluaremos el modelo

In [17]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8758 - loss: 0.8452
Test Loss: 0.8896362781524658
Test Accuracy: 0.8825107216835022


Después haremos predicciones utilizando nuestro modelo

In [19]:
# Make predictions
y_pred_dnn = model.predict(X_test)
y_pred_dnn = (y_pred_dnn > 0.5).astype(int) # Convert probabilities to class labels

59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [22]:

# Calculate and print evaluation metrics
conf_matrix_dnn = confusion_matrix(y_test, y_pred_dnn)
print(f"Confusion Matrix (DNN):\n{conf_matrix_dnn}")

f1_dnn = f1_score(y_test, y_pred_dnn)
print(f"F1 Score (DNN): {f1_dnn}")

roc_auc_dnn = roc_auc_score(y_test, model.predict(X_test))
print(f"ROC AUC Score (DNN): {roc_auc_dnn}")

Confusion Matrix (DNN):
[[1557   40]
 [ 179   88]]
F1 Score (DNN): 0.44556962025316454
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
ROC AUC Score (DNN): 0.823838705062629
